# Flexible MTL design for testing different configurations


# Set up

First, we need to clone the data. Instead of manually uploading, please clone the repository from Yipeng by running:

`!git clone -b oxpet --single-branch https://weisslab.cs.ucl.ac.uk/WEISSTeaching/datasets.git`

In [1]:
!git clone -b oxpet --single-branch https://weisslab.cs.ucl.ac.uk/WEISSTeaching/datasets.git

fatal: destination path 'datasets' already exists and is not an empty directory.


Then, upload the `library.zip` file. This is a zipped version of the entire library, for is a manual process at the moment, until we work on a better way of exporting our library.

Please not that torchvision.models.utils is not supported on the environment run here. A manual fix has been added to patch this. This should be visible when you run the code. Once uploaded, unzip the library using `unzip  library.zip`

In [2]:
!unzip -o flexible_mtl.zip 

Archive:  flexible_mtl.zip
  inflating: requirements.txt        
  inflating: legacy/__init__.py      
  inflating: legacy/utils.py         
  inflating: legacy/train.py         
  inflating: legacy/main.py          
  inflating: legacy/models/bodys.py  
  inflating: legacy/models/densenet.py  
  inflating: legacy/models/__init__.py  
  inflating: legacy/models/model.py  
  inflating: legacy/models/resnet.py  
  inflating: legacy/models/utils.py  
  inflating: legacy/models/heads.py  
  inflating: legacy/__pycache__/__init__.cpython-38.pyc  
  inflating: legacy/__pycache__/train.cpython-38.pyc  
  inflating: legacy/__pycache__/utils.cpython-38.pyc  
  inflating: legacy/__pycache__/utils.cpython-37.pyc  
  inflating: legacy/__pycache__/train.cpython-37.pyc  
  inflating: legacy/__pycache__/__init__.cpython-37.pyc  
  inflating: legacy/criterion/loss_functions.py  
  inflating: legacy/criterion/metric_functions.py  
  inflating: legacy/criterion/criterion.py  
  inflating: legacy/data/da

Now we must install the relevant librries from the requirements file (this should be included in library.zip). Do this by running the following:
`!python3 -m venv env`

`!source env/bin/activate`

`!pip install -r requirements.txt`

You may get an error that the latest version of numpy could not be found. You can install this manually, although I believe it should exist by defualt in the first palce.

In [3]:
!python3 -m venv mtl_test_colab
!source mtl_test_colab/bin/activate
!which python
!pip install -r requirements.txt

Error: Command '['/content/mtl_test_colab/bin/python3', '-Im', 'ensurepip', '--upgrade', '--default-pip']' returned non-zero exit status 1.
/bin/bash: mtl_test_colab/bin/activate: No such file or directory
/usr/local/bin/python
  Using cached fonttools-4.28.5-py3-none-any.whl (890 kB)
  Using cached h5py-3.6.0-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (4.1 MB)
  Using cached imageio-2.13.5-py3-none-any.whl (3.3 MB)
  Using cached matplotlib-3.5.1-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.whl (11.2 MB)
ERROR: Could not find a version that satisfies the requirement numpy==1.22.0 (from versions: 1.3.0, 1.4.1, 1.5.0, 1.5.1, 1.6.0, 1.6.1, 1.6.2, 1.7.0, 1.7.1, 1.7.2, 1.8.0, 1.8.1, 1.8.2, 1.9.0, 1.9.1, 1.9.2, 1.9.3, 1.10.0.post2, 1.10.1, 1.10.2, 1.10.4, 1.11.0, 1.11.1, 1.11.2, 1.11.3, 1.12.0, 1.12.1, 1.13.0rc1, 1.13.0rc2, 1.13.0, 1.13.1, 1.13.3, 1.14.0rc1, 1.14.0, 1.14.1, 1.14.2, 1.14.3, 1.14.4, 1.14.5, 1.14.6, 1.15.0rc1, 1.15.0rc2, 1.15.0, 1.15.1, 1.15.2, 1.15.3, 1.15

## Now... you design and run the model of your choice!

```diff
!PLEASE read before running 
```
This notebook is different than the ones I've sent previously. The major difference is that it allows you to quickly and easily design the MTL model of your choice with ease, without needing to change the backend code. It also has added support for storing loss values, validation data, testing data and also the use of custom metrics and custom losses.

Please note, it does NOT currently support saving and loading of models. That is a work in progress, but I've put it as a low priority task since it seems like our models are running quickly anyways. If there are urgent issues, then I will get it done ASAP.

Please read the instructions here as they will give you a good understanding of how this is intended to be used. Run experiments on it, and if there are any issues notify me and create them on GitHub.

In [1]:
# libraries

# GLOBALS

TRAIN_BATCH_SIZE = 8
TEST_BATCH_SIZE = 8
VAL_BATCH_SIZE = 8

TASKS = ['seg', 'class', 'bb']

## Loading the data

It is good practice to always laod the data first. The dataloader has been modified to take in a path input from the user, to make it more flexible.

In [2]:
## load data libraries
from data.data import OxpetDataset, fast_loader

## get your datasets

data_path = 'datasets/data_new/' # point to the directory where the data sits
trainset = OxpetDataset(data_path + 'train', TASKS)
testset = OxpetDataset(data_path + 'test', TASKS)
valset = OxpetDataset(data_path + 'val', TASKS)

## define your dataloaders


trainloader = fast_loader(trainset, batch_size=TRAIN_BATCH_SIZE)
testloader = fast_loader(trainset, batch_size=TEST_BATCH_SIZE)
valloader = fast_loader(trainset, batch_size=VAL_BATCH_SIZE)

# Defining the model

In [9]:
## imports
from torch.optim import Adam
from models.utils import get_prebuilt_model

MODEL, LOSS = get_prebuilt_model('resnet34', decoders='+'.join(TASKS),
                                 losses=f'DiceLoss+CrossEntropyLoss+{TRAIN_BATCH_SIZE/10000}*L1Loss', weights='uniform::1'
                                 )
OPTIMIZER = Adam(MODEL.parameters(), lr=1e-04)

In [10]:
from run import RunTorchModel
from criterion.metric_functions import MultiAccuracy, PixelAccuracy, Recall, Precision, F1Score, Jaccard

# initialise run, this is analogous to model.compile() in keras
run_instance = RunTorchModel(
    model=MODEL, optimizer=OPTIMIZER, loss=LOSS, metrics={
        'class':[MultiAccuracy()],
        'seg': [MultiAccuracy(), PixelAccuracy(), Recall(), Precision(), F1Score(), Jaccard()]
        }
)

# training params
EPOCHS=2
VERBOSE=3
TRACK_HISTORY=True # to save model hist

run_instance.train(trainloader, valloader=valloader, epochs=EPOCHS, verbose=VERBOSE, track_history=TRACK_HISTORY) # analogous to model.fit() in keras
run_instance.get_history() # training history

CUDA device detected. Running on GPU.
Training model...
EPOCH 1 STARTED
---------------
Batch 1 complete. Time taken: load(0.377), train(0.443), total(0.82). 
RandomCombinedLoss(0.401), seg(0.154), class(0.206), bb(0.041)
MultiAccuracy(3.14e+05), PixelAccuracy(0.506), Recall(0.637), Precision(0.424), F1Score(0.509), Jaccard(0.339)
Batch 2 complete. Time taken: load(0.388), train(0.339), total(0.726). 
RandomCombinedLoss(0.432), seg(0.143), class(0.252), bb(0.0371)
MultiAccuracy(3.14e+05), PixelAccuracy(0.498), Recall(0.657), Precision(0.421), F1Score(0.513), Jaccard(0.332)
Batch 3 complete. Time taken: load(0.744), train(0.338), total(1.08). 
RandomCombinedLoss(0.436), seg(0.15), class(0.254), bb(0.0325)
MultiAccuracy(3.2e+05), PixelAccuracy(0.474), Recall(0.657), Precision(0.396), F1Score(0.493), Jaccard(0.311)
Batch 4 complete. Time taken: load(0.384), train(0.335), total(0.719). 
RandomCombinedLoss(0.468), seg(0.153), class(0.284), bb(0.0304)
MultiAccuracy(3.22e+05), PixelAccuracy(0

KeyboardInterrupt: ignored